In [1]:
import torch
import torch.nn as nn
import clip
from PIL import Image
import pandas as pd
import requests
import os.path as osp
import pickle
import random
import numpy as np
from pathlib import Path
import sys
from operator import itemgetter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
dataset_path = "C:/Users/aphri/Documents/t0002/work/data/w210_data/target_store_furniture_datasets.csv"
image_storage = "C:/Users/aphri/Documents/t0002/work/data/w210_data/target_images"
pickle_path = "C:/Users/aphri/Documents/t0002/work/data/w210_data/pickle"
model_path = "C:/Users/aphri/Documents/t0002/work/data/w210_data/model"

Path(image_storage).mkdir(parents=True, exist_ok=True)
Path(pickle_path).mkdir(parents=True, exist_ok=True)
Path(model_path).mkdir(parents=True, exist_ok=True)

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [5]:
d1 = pd.read_csv(dataset_path)

In [6]:
d1.head()

,title,url,brand,main_image,sku,description,raw_description,gtin13,currency,price,...,sub_category_1,sub_category_2,sub_category_3,images,raw_specifications,specifications,highlights,raw_highlights,uniq_id,scraped_at
0,Salinas Mission Desk & Hutch Antique White - B...,https://www.target.com/p/salinas-mission-desk-...,Bush Furniture,https://target.scene7.com/is/image/Target/GUES...,50393086,The Bush Furniture Salinas Small Computer Desk...,"<div class=""h-margin-v-default"" data-test=""ite...",42976502997,USD,449.99,...,Home Office Furniture,Desks,NaN,https://target.scene7.com/is/image/Target/GUES...,"<div class=""styles__StyledCol-sc-ct8kx6-0 iKGd...",Dimensions (Overall): 62.87 Inches (H) x 47.16...,Compact 48W x 23D design presents an ideal com...,"<li class=""styles__Bullet-sc-6aebpn-0 eIfLaI"">...",c456e14d-0693-5552-8f90-eead4a0f1f50,2022-06-12
1,Finish Dresser with 6 Drawers White - Benzara,https://www.target.com/p/finish-dresser-with-6...,Benzara,https://target.scene7.com/is/image/Target/GUES...,78671604,You can have extra storage space for your bedr...,"<div class=""h-margin-v-default"" data-test=""ite...",842822178709,USD,307.49,...,Bedroom Furniture,Dressers,NaN,https://target.scene7.com/is/image/Target/GUES...,"<div class=""styles__StyledCol-sc-ct8kx6-0 iKGd...",Number of Pieces: 1 | Number of Pieces: 1 | Di...,Elegant dresser has 6 drawers with hollow meta...,"<li class=""styles__Bullet-sc-6aebpn-0 eIfLaI"">...",b020d91e-77ab-5c3e-b1e6-2fe01abe41b6,2022-06-12
2,Full Arcadia Nailbutton Metallic Upholstered B...,https://www.target.com/p/full-arcadia-nailbutt...,Skyline Furniture,https://target.scene7.com/is/image/Target/GUES...,49158084,Clean lines and lustrous nailheads give this u...,"<div class=""h-margin-v-default"" data-test=""ite...",840966280517,USD,807.99,...,Bedroom Furniture,Beds,NaN,https://target.scene7.com/is/image/Target/GUES...,"<div class=""styles__StyledCol-sc-ct8kx6-0 iKGd...",Dimensions (Overall): 51.0 Inches (H) x 50.0 I...,"To find the look that is right for you, order ...","<li class=""styles__Bullet-sc-6aebpn-0 eIfLaI"">...",d0972ad8-c10a-5d84-86a1-4d25cd361229,2022-06-12
3,Broadway Side Cabinet Black - Lifestorey,https://www.target.com/p/broadway-side-cabinet...,Lifestorey,https://target.scene7.com/is/image/Target/GUES...,80273366,Enhance the look of your room with the Broadwa...,"<div class=""h-margin-v-default"" data-test=""ite...",24319119024,USD,249.99,...,Living Room Furniture,Storage Furniture,NaN,https://target.scene7.com/is/image/Target/GUES...,"<div class=""styles__StyledCol-sc-ct8kx6-0 iKGd...",Dimensions (Overall): 34 Inches (H) x 19 Inche...,Includes one (1) side cabinet with quatrefoil ...,"<li class=""styles__Bullet-sc-6aebpn-0 eIfLaI"">...",dade134d-f251-5120-97be-23eb1e01c0f8,2022-06-12
4,2pk Backpback Lawn Chairs with Silver Frame - ...,https://www.target.com/p/2pk-backpback-lawn-ch...,Life is Good,https://target.scene7.com/is/image/Target/GUES...,85644896,Keep it simple with this sturdy and dependable...,"<div class=""h-margin-v-default"" data-test=""ite...",722938140978,USD,279.99,...,Patio Furniture,Patio Chairs,NaN,https://target.scene7.com/is/image/Target/GUES...,"<div class=""styles__StyledCol-sc-ct8kx6-0 iKGd...",Number of Pieces: 2 | Number of Pieces: 2 | Se...,Set of two | Rustproof aluminum frame | Remova...,"<li class=""styles__Bullet-sc-6aebpn-0 eIfLaI"">...",f2e0d558-e93f-541b-98b1-6fff36f1dc89,2022-06-12


In [7]:
d1.columns

Index(['title', 'url', 'brand', 'main_image', 'sku', 'description',
       'raw_description', 'gtin13', 'currency', 'price', 'availability',
       'availableDeliveryMethod', 'available_branch', 'primary_category',
       'sub_category_1', 'sub_category_2', 'sub_category_3', 'images',
       'raw_specifications', 'specifications', 'highlights', 'raw_highlights',
       'uniq_id', 'scraped_at'],
      dtype='object')

In [8]:
d2 = d1[["uniq_id", "sub_category_1", "sub_category_2", "sub_category_3"]]

In [12]:
np.unique(d2["sub_category_1"].dropna())

array(['Accessories', 'Action Figures & Playsets', 'Amplifiers & Effects',
       'Automotive', 'Baby Safety', 'Bar & Wine', 'Bathroom Furniture',
       'Bedroom Furniture', 'Boards & Easels', 'Books', 'Camping Gear',
       'Cat Supplies', 'Christmas', 'Cleaning Supplies',
       'Decor Style Ideas', 'Desk Organization', 'Dog Supplies',
       'Dolls & Dollhouses', 'Dorm Furniture', 'Dress Up & Pretend Play',
       'Entryway Furniture & Organization', 'Fall Lawn Cleanup',
       'Fire Pits & Patio Heaters', 'Furniture Deal of the Day',
       'Furniture Ways to Shop', 'Heating, Cooling & Air Quality',
       'Home Appliances', 'Home Deals', 'Home Decor', 'Home Improvement',
       'Home Office Furniture', "Kids' Crafts", "Kids' Home",
       'Kitchen & Dining Furniture', 'Kitchen & Table Linens',
       'Laundry Care', 'Lawn & Garden', 'Living Room Furniture', 'Music',
       'Nintendo Switch', 'Nursery', 'Outdoor Decor', 'Outdoor Dining',
       'Outdoor Recreation', 'Outdoor Shade

In [13]:
len(d2["sub_category_1"].dropna())

42078

In [14]:
len(d2["sub_category_2"].dropna())

41525

In [15]:
len(d2["sub_category_3"].dropna())

5118